In [ ]:
import re
import numpy as np
import nltk
import zlib
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
pattern_url = re.compile(r'http[s]?://lenta.ru/\w+/[0-9]{4}/[0-9]{2}/[0-9]{2}/\w+')

In [ ]:
def open_file(index):
    f = open('dataset/lenta.ru' + index, encoding = 'utf-8', errors = 'ignore')
    return f.read().lower()

In [ ]:
def url_text(raw_text):
    clean_text = list()
    keys = pattern_url.findall(raw_text)
    text = pattern_url.split(raw_text)

    clean_text = [re.findall(r'\w+',article) for article in text]

    return dict(zip(keys,clean_text[1:]))

In [ ]:
def merge_dicts(dict1, dict2):
    return {**dict1, **dict2}

In [ ]:
def get_merged_straight_index():
    straight_index = dict()
    for i in range(1,9):
        raw_txt = open_file(str(i))
        straight_index = merge_dicts(straight_index, url_text(raw_txt))
    return straight_index

In [ ]:
def url_base():
    base_of_url = dict()
    urls = list()
    for i in range(1,8):
        f = open_file(str(i))
        urls += pattern_url.findall(f)
        
    for counter, url in enumerate(urls):
        base_of_url = merge_dicts(base_of_url, {counter + 1 : url})
    return base_of_url

In [ ]:
#Кодировка Фибоначчи
def fib_encode(num):
    fib_list = [1,1]
    fib_ids = []
    code = 0
    
    while num >= fib_list[-1]:
        fib_list.append(fib_list[-2]+fib_list[-1])
        
    fib_list.reverse()
    
    for counter, i in enumerate(fib_list):
        if(num >= fib_list[counter]):
            num -= fib_list[counter]
            fib_ids.append(len(fib_list)-counter-1)

    fib_ids.reverse()
    
    for i in fib_ids:
        mask = 1
        mask <<= (i-1)
        code |= mask
        
    return code

In [ ]:
#Конкатенация 2х чисел Фибоначчи
def len_fib(fib):
    len_ = 0
    while fib:
        fib >>= 1
        len_ += 1
    
    return len_

def fib_concat(fib1, fib2):
    fib1 <<= 1
    fib1 += 1
    fib1 <<= len_fib(fib2)
    fib1 |= fib2
    
    return fib1

In [ ]:
def encode_list(list_):
    res = list_[0]
    for i in list_[1:]:
        res = fib_concat(res,i)
    return res

In [ ]:
def get_n_fib(num):
    return round((((1+5**(1/2))/2)**(num+1))/5**(1/2))

In [ ]:
# Декодировка Фибоначчи
def decode_fib(bits):
    flag = 0
    counter = 0
    sum_ = 0
    res = list()

    
    while bits:
        counter += 1
        if bits & 1:
            flag += 1
            if flag > 1:
                flag = 0
                counter = 0
                res.append(sum_)
                sum_ = 0
            else:
                sum_ += (get_n_fib(counter))
              
        else:
            flag = 0
            
        bits >>= 1
    res.append(sum_)
    res.reverse()
    return res

In [ ]:
#Составление списка словарей частот: {DOC_ID: term_frequency}
def doc_frequency(base_url_dict):
    posting_list = list()
    for url in base_url_dict:
        counts = Counter(straight_index[base_url_dict[url]])
        posting_list.append((doc_id, dict(counts.most_common())))
    return posting_list

In [ ]:
def unique_words(urls):
    raw_txt = set()
    for doc_id in urls:
        raw_txt |= set(straight_index[base_url_dict[doc_id]])
    return raw_txt

In [ ]:
def get_text_corpus(nums):
    corpus = list()
    for i in nums:
        f = open_file(str(i))
        text = pattern_url.split(f)
        corpus += [re.sub(r'\W+', ' ', article) for article in text[1:]]

    return corpus

In [ ]:
def make_shift(list_of_docs):
    res = [list_of_docs[i] - list_of_docs[i-1] for i in range(1, len(list_of_docs))]
    return [list_of_docs[0]] + res

def shift_away(list_of_docs):
    res = list()
    res.append(list_of_docs[0])
    for i in range(1, len(list_of_docs)):
        res += [res[i-1] + list_of_docs[i]]
    return res

In [ ]:
def get_compressed_index(corpus):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    arr = X.toarray()
    d = dict()
    for term in vectorizer.get_feature_names():
        term_id = vectorizer.vocabulary_.get(term)
        docs = np.where(arr[:,term_id]>0)
        freq = arr[:,term_id][np.where(arr[:,term_id]>0)[0]]
        tmp = make_shift(docs[0])        
        fib_docs = make_compression(tmp)
        fib_freq = make_compression(freq)
        
        d[zlib.compress(str.encode(term))] = (encode_list(fib_docs), encode_list(fib_freq))
    
    return d

In [ ]:
def make_compression(docs):
    fib_list = list()

    for i in docs:
        fib_list.append(fib_encode(i))
        
    return fib_list

In [ ]:
# Получили поисковый индекс, получили базу URL
INVERTED_COMPRESSED_INDEX = get_compressed_index(get_text_corpus([1,2,3,4,5,6,7]))

In [ ]:
URL_BASE = url_base()

In [ ]:
def search_one_term(term):
    l = list()
    
    term = term.lower()
    t_zlib = zlib.compress(str.encode(term))
    
    try:
        docs_fib, freq_fib = INVERTED_COMPRESSED_INDEX[t_zlib]
    except KeyError:
        print('Нет совпадений со словом:', term)
        return [],[]
    
    return shift_away(decode_fib(docs_fib)), decode_fib(freq_fib)

In [ ]:
def extract_request(request):
    res = request
    
    res = re.sub(r'&', r' & ', res)
    res = re.sub(r'\|', r' | ', res)
    res = re.sub(r'\(', r' ( ', res)
    res = re.sub(r'\)', r' ) ', res)
    res = res.split()
    
    return res

In [ ]:
def check_querry(q_list):
    c_brackets = 0
    c_words_operators = 0
    
    for i in q_list:
        
        if i == '(':
            c_brackets += 1
        elif i == ')':
            c_brackets -= 1
        elif i in ['&', '|']:
            c_words_operators -= 1
        else:
            c_words_operators += 1
            
    if c_brackets:
        return -1
    elif c_words_operators != 1:
        return -2
    else:
        return 0

In [ ]:
def boolean_search(request):
    request = extract_request(request)
    if not len(request):
        print('Задан пустой поисковый запрос')
        return
    l = check_querry(request)
    if l == -1:
        print("Дисбаланс скобок в запросе")
        return
    if l == -2:
        print("Неверное количество операторов или операндов в запросе")
        return
    stack = list()
    out_s = list()
    d = dict()
    
    for i in request:
        if i == '(':
            stack.append(i)
        elif i == ')':
            while stack[-1] != '(':
                out_s.append(stack.pop())
            stack.pop()
        elif i == '|':
            while stack and stack[-1] == '&':
                out_s.append(stack.pop())
            stack.append(i)
        elif i == '&':
            stack.append(i)
        else:
            new_docs, freq = search_one_term(i)
            for i in range(len(new_docs)):
                if(not d.get(new_docs[i])):
                    d[new_docs[i]] = freq[i]
                else:
                    d[new_docs[i]] += freq[i]
                
            out_s.append(new_docs)
            
    while stack:
        out_s.append(stack.pop())

    c = 0
    while len(out_s) > 1:
        docs = []
        if out_s[c] == '&':
            docs += list(set(out_s[c-2]) & set(out_s[c-1]))
            out_s = out_s[:c-2] + [docs] + out_s[c+1:]
            c-=2
            
        elif out_s[c] == '|':
            docs += list(set(out_s[c-2]) | set(out_s[c-1]))
            out_s = out_s[:c-2] + [docs] + out_s[c+1:]
            c-=2
            
        else:
            pass
        
        c += 1
    final  = []
    for i in out_s[0]:
        final.append((URL_BASE[i+1], d[i]))
    if not len(final):
        print('По вашему запросу ничего не найдено, попробуйте ещё')
    else:
        final.sort(key = lambda x:x[1], reverse=True)
        print('Итого {} стр:\n'.format(len(final)))
        for i in range(len(final)):
            print('URL: {}\nВхождений слов из запроса{:_>20}\n'.format(final[i][0], final[i][1]))

    return

In [ ]:
boolean_search('')

Задан пустой поисковый запрос


In [ ]:
boolean_search('путин & | медведев')

Неверное количество операторов или операндов в запросе


In [ ]:
boolean_search('((путин) & лес)))')

Дисбаланс скобок в запросе


In [ ]:
boolean_search('аввап & путин')

Нет совпадений со словом: аввап
По вашему запросу ничего не найдено, попробуйте ещё


In [ ]:
boolean_search('путин & медведев&сша')

Итого 9 стр:

URL: http://lenta.ru/russia/2001/11/01/malyshev
Вхождений слов из запроса__________________23

URL: http://lenta.ru/culture/2001/07/19/archer
Вхождений слов из запроса__________________17

URL: http://lenta.ru/online/2009/05/08/rususa
Вхождений слов из запроса__________________17

URL: http://lenta.ru/news/2015/10/15/otkaz
Вхождений слов из запроса__________________13

URL: http://lenta.ru/news/2009/04/27/soldier
Вхождений слов из запроса__________________12

URL: http://lenta.ru/articles/2008/12/31/finalblogs
Вхождений слов из запроса___________________5

URL: http://lenta.ru/news/2015/06/19/williams
Вхождений слов из запроса___________________4

URL: http://lenta.ru/articles/2010/03/30/review
Вхождений слов из запроса___________________4

URL: http://lenta.ru/news/2009/12/28/an124
Вхождений слов из запроса___________________3



In [ ]:
boolean_search('путин&налоги')

Итого 4 стр:

URL: http://lenta.ru/articles/2008/12/04/putin
Вхождений слов из запроса__________________50

URL: http://lenta.ru/russia/2001/11/01/malyshev
Вхождений слов из запроса__________________20

URL: http://lenta.ru/articles/2015/12/30/sotsialka
Вхождений слов из запроса___________________6

URL: http://lenta.ru/news/2012/04/24/tax
Вхождений слов из запроса___________________4

